In [ ]:
import pandas as pd
import dgl
import torch as th
import numpy as np
import networkx as nx
import pickle

# Load data

In [ ]:
emb = np.load("emb.npy")

# Heterogeneous Graph Construction

In [ ]:
df_edges['edge_full_type'].unique()[0:-1]

In [ ]:
node_sum = 0
for etype in df_edges['edge_full_type'].unique()[0:-1]:
    num = df_edges[df_edges.edge_full_type == etype].edge.nunique()
    node_sum += num
    print("node", etype, num)
    print("edge", etype, df_edges[df_edges.edge_full_type == etype].shape[0])
print(node_sum)

In [ ]:
edge2type = {}

mapping_node = {v:u+len(mapping_cust) for u,v in enumerate(df_edges_het.edge.unique())}
mapping_edge_type = {v:u for u,v in enumerate(df_edges['edge_full_type'].unique()[0:-1])}

edge_src = list(map(lambda x:mapping_cust[x], df_edges_het.cust_id.to_numpy()))
edge_dst = list(map(lambda x:mapping_node[x], df_edges_het.edge.to_numpy()))
edge_type = list(map(lambda x:mapping_edge_type[x], df_edges_het.edge_full_type.to_numpy()))

In [ ]:
txn_set = set()

txn_src = df_edges_trx.src_value.to_numpy()
txn_dst = df_edges_trx.dst_value.to_numpy()
for index, txn in enumerate(txn_src):
    if txn in mapping_cust and txn_dst[index] in mapping_cust:
        cust_src = mapping_cust[txn]
        cust_dst = mapping_cust[txn_dst[index]]
        if (cust_src, cust_dst) not in txn_set and (cust_dst, cust_src) not in txn_set:
            edge_src.append(cust_src)
            edge_dst.append(cust_dst)
            edge_type.append(len(mapping_edge_type))
            txn_set.add((cust_src, cust_dst))

In [ ]:
edge_src_bid = edge_src + edge_dst
edge_dst_bid = edge_dst + edge_src
edge_type_bid = edge_type + [etype + len(mapping_edge_type) + 1 for etype in edge_type]

In [ ]:
for src, dst, etype in zip(edge_src_bid, edge_dst_bid, edge_type_bid):
    edge2type[(src,dst)] = etype

# Clustering for Abnormal Patterns

In [ ]:
import collections
from typing import List, Set, Dict, Union

import numpy as np
import tqdm
from scipy import sparse as sp
from sklearn.cluster import KMeans

from sklearn.metrics.pairwise import euclidean_distances

def get_comm_feature(nodes: List[int], scores: np.ndarray) -> np.ndarray:
    """
    Compute the distribution of edge types as the community feature."
    """
    nodes = set(nodes)
    arr = np.mean([scores[u] for u in nodes],axis=0)
    return arr


def get_patterns(comms: List[List[int]],
                 node_labels: np.ndarray, n_patterns: int) -> (np.ndarray, List[List[int]], np.ndarray):
    """
    Compute community features and use k-means to get patterns' features, size distributions, and supports.
    """
    comm_features = [get_comm_feature(nodes, node_labels) for nodes in comms]
    k_means = KMeans(n_patterns)
    comm_labels = k_means.fit_predict(comm_features)
    pattern_features = k_means.cluster_centers_
    pattern_sizes = [[] for _ in range(n_patterns)]
    pattern_support = np.zeros(n_patterns)
    for i, label in enumerate(comm_labels):
        pattern_support[label] += 1
        pattern_sizes[label].append(len(comms[i]))
    return pattern_features, pattern_sizes, pattern_support


def compute_node_pattern_score(pattern_features: np.ndarray,
                               adj_mat: sp.spmatrix,
                               neighbors: Dict[int, Set[int]],
                               scores:np.ndarray) -> np.ndarray:
    """
    Scoring nodes based on local structures.
    """

    n_nodes = adj_mat.shape[0]
    # score of center node
    node_local_features = np.array(scores, dtype=np.float64)
    for u in tqdm.tqdm(range(n_nodes), desc='NodeLocalFeature'):
        node_local_features[u] += np.sum([scores[v] for v in neighbors[u]], axis=0)
    # First Order: Pass 1 in the paper
    # @ 矩阵-向量乘法
    node_first_order_scores = euclidean_distances(node_local_features, pattern_features)
    # Second Order: Pass 2 in the paper
    deg_vec = np.array(adj_mat.sum(1)).squeeze()
    # diags 提取对角线或构造对角线数组。
    node_second_order_scores = sp.diags((adj_mat @ deg_vec) ** -1) @ adj_mat @ (
            deg_vec[:, None] * node_first_order_scores)
    node_pattern_scores = node_first_order_scores + node_second_order_scores
    return node_pattern_scores

In [ ]:
emb = np.load("emb.npy")

In [ ]:
comm_features = [get_comm_feature(nodes, emb) for nodes in new_groups]

In [ ]:
np.save("comm_features.npy", comm_features)

In [ ]:
SSE = []  
for k in range(2,20):
    estimator = KMeans(n_clusters=k)  
    estimator.fit(comm_features)
    SSE.append(estimator.inertia_) 
X = range(2,20)
plt.xlabel('k')
plt.ylabel('SSE')
plt.plot(X,SSE,'o-')
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score   
scores = []
models = []

for i in range(2, 20):
    cluster = KMeans(n_clusters=i).fit(comm_features)
    y_pred = cluster.labels_
    centroid = cluster.cluster_centers_
    silhouette_avg = silhouette_score(comm_features, y_pred)
    print(i, silhouette_avg)

In [ ]:
estimator = KMeans(n_clusters=5)  # 构造聚类器
estimator.fit(comm_features)
# print(estimator.labels_)
# squared distance to cluster center
dist = estimator.transform(comm_features)

center_dists = np.array([dist[i][x] for i,x in enumerate(estimator.labels_)])

In [ ]:
tsne[estimator.labels_ == 0]

In [ ]:
label = estimator.labels_

r = pd.Series(estimator.labels_, index = range(len(comm_features))) 
r.columns = [u'聚类类别']

from sklearn.manifold import TSNE
tsne = TSNE()
tsne.fit_transform(comm_features) 
tsne = pd.DataFrame(tsne.embedding_, index = range(len(comm_features)))

import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False 

d = tsne[label == 0]     
plt.plot(d[0], d[1], 'r.')
d = tsne[label == 1]
plt.plot(d[0], d[1], 'go')
d = tsne[label == 2]
plt.plot(d[0], d[1], 'b*')
d = tsne[label == 3]
plt.plot(d[0], d[1], 'y*')
d = tsne[label == 4]
plt.plot(d[0], d[1], 'k*')
plt.show()

# Abnomral Group Search and Generation

In [ ]:
df_acct_status = pd.read_csv('/home/binwu5/acct_status.csv')
cust_label = dict(zip(df_acct_status['cust_id'],df_acct_status['acct_rstrd_y_n']))

In [ ]:
new_groups = np.load("new_groups.npy", allow_pickle=True)

In [ ]:
pattern_features, pattern_sizes, pattern_support = get_patterns(new_groups, emb, 5)

In [ ]:
np.save("new_groups.npy", new_groups)

In [ ]:
pattern_p = pattern_support / pattern_support.sum()

In [ ]:
adj_mat = nx.to_scipy_sparse_matrix(nx_g)

In [ ]:
import scipy.sparse
# scipy.sparse.save_npz('adj_mat.npz', adj_mat)
adj_mat = scipy.sparse.load_npz('adj_mat.npz')

In [ ]:
node_local_features = np.zeros_like(emb)

for u in tqdm.tqdm(range(len(emb)), desc='NodeLocalFeature'):
    node_local_features[u] = np.mean([emb[v] for v in (list([u]) + list([n for n in nx_g.neighbors(u)]))], axis=0)

node_first_order_scores = euclidean_distances(node_local_features, pattern_features)

In [ ]:
from scipy import sparse as sp

# Second Order: Pass 2 in the paper
deg_vec = np.array(adj_mat.sum(1)).squeeze()
# diags 提取对角线或构造对角线数组。
# node_second_order_scores = sp.diags((adj_mat @ deg_vec) ** -1) @ adj_mat @ (
#         deg_vec[:, None] * node_first_order_scores)

node_degree_sum = adj_mat @ deg_vec
node_score = adj_mat @ (deg_vec[:, None] * node_first_order_scores)

node_pattern_scores = np.zeros_like(node_first_order_scores)

for u in tqdm.tqdm(range(len(emb)), desc='NodeSecondFeature'):
    if deg_vec[u] == 0:
        node_pattern_scores[u] = node_first_order_scores[u] + 100
    else:
        node_pattern_scores[u] = node_first_order_scores[u] + node_score[u] / node_degree_sum[u]

In [ ]:
node_pattern_scores = node_first_order_scores

In [ ]:
n_node = len(emb)

In [ ]:
# used_seeds = set()
node_degrees = np.zeros(n_cust)

for u in tqdm.tqdm(range(n_cust)):
    group = set([u])
    for v in list([n for n in nx_g.neighbors(u)]):
        if v >= n_cust:
            group = group | set([n for n in nx_g.neighbors(v)])
        else:
            group.add(v)
    node_degrees[u] = len(group)

degree_node_dict = collections.defaultdict(list)
    
for i, d in enumerate(node_degrees):
    degree_node_dict[d].append(i)

In [ ]:
node_degrees = np.array(adj_mat.sum(1)).squeeze().astype(int)
degree_node_dict = collections.defaultdict(list)
for i, d in enumerate(node_degrees):
    degree_node_dict[d].append(i)
    
pattern_degree_seeds = [{d: sorted(nodes, key=lambda i: -x[i])
                                      for d, nodes in degree_node_dict.items()}
                                     for x in node_pattern_scores.T]

In [ ]:
def get_seed(target_size: int, degree_seeds: Dict[int, List[int]],
             used_seeds: Set[int], eps: int = 5) -> Union[int, None]:
    """
    Find the best seed that has never be picked before.
    """
    for deg in range(target_size - 1, target_size + eps):
        sorted_seeds = degree_seeds.get(deg, [])
        # print(deg)
        if len(sorted_seeds) == 0:
            continue
        while len(sorted_seeds):
            seed = sorted_seeds.pop()
            if seed not in used_seeds:
                used_seeds.add(seed)
                return seed
    else:
        return None

In [ ]:
def sample():
    n_try = 0
    while (n_try < 20) and (len(used_seeds) < n_cust):
        n_try += 1
        pattern_id = np.random.choice(len(pattern_p), p=pattern_p)
        target_size = np.random.choice(pattern_sizes[pattern_id])
        seed = get_seed(target_size, pattern_degree_seeds[pattern_id], used_seeds)
        if seed is None:
            continue
#         group = set([seed])
#         for v in list([n for n in nx_g.neighbors(seed)]):
#             if v >= n_cust:
#                 group = group | set([n for n in nx_g.neighbors(v)])
#                 for node in set([n for n in nx_g.neighbors(v)]):
#                     if node < n_cust:
#                         used_seeds.add(node)
#             else:
#                 group.add(v)
#                 used_seeds.add(v)
        # print("target_size:", target_size)
        # print("size:", len(group))
        
        comm = [seed] + list(nx_g.neighbors(seed))
        for node in comm:
            used_seeds.add(node)
        
        return comm, pattern_id, target_size
    
        # return group, pattern_id, target_size
    else:
        raise ValueError('(Almost) Run out of seeds!')

In [ ]:
used_seeds = set()
pattern_degree_seeds = np.load("pattern_degree_seeds.npy", allow_pickle = True)

for group in new_groups:
    for node in group:
        used_seeds.add(node)

pred_comms = []
out_comms = []
n_comms = 1500

try:
    for i in tqdm.tqdm(range(n_comms)):
#         comm, pattern_id, target_size = sample()
#         pred_comms.append(comm)
#         custNum = 0
#         for node in comm:
#             if node < n_cust:
#                 custNum += 1
#         out_comms.append([i, pattern_id, node_pattern_scores[comm[0]][pattern_id], len(comm), custNum])
        pred_comms.append(beam_sample(pattern_features, emb))
except ValueError as e:
    print('Warning!!!', e)